# Machine translation


We will be comparing the performance of three different architectures:
* A vanilla RNN
* A GRU-RNN
* A transformer

Do not forget to **select the runtime type as GPU!**

**Sources**

* Dataset: [Tab-delimited Bilingual Sentence Pairs](http://www.manythings.org/anki/)

<!---
M. Cettolo, C. Girardi, and M. Federico. 2012. WIT3: Web Inventory of Transcribed and Translated Talks. In Proc. of EAMT, pp. 261-268, Trento, Italy. pdf, bib. [paper](https://aclanthology.org/2012.eamt-1.60.pdf). [website](https://wit3.fbk.eu/2016-01).
-->

* The code is inspired by this [pytorch tutorial](https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html).

*This notebook is quite big, use the table of contents to easily navigate through it.*

# Imports and data initializations

We first download and parse the dataset. From the parsed sentences
we can build the vocabularies and the torch datasets.
The end goal of this section is to have an iterator
that can yield the pairs of translated datasets, and
where each sentences is made of a sequence of tokens.


## Imports

In [ ]:

!python3 -m spacy download en > /dev/null
!python3 -m spacy download fr > /dev/null
!pip install torchinfo > /dev/null
!pip install einops > /dev/null
!pip install wandb > /dev/null

from itertools import takewhile
from collections import Counter, defaultdict

import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, Vocab
from torchtext.datasets import IWSLT2016

import einops
import wandb
from torchinfo import summary

2023-04-11 05:06:56.605538: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-11 05:06:56.663712: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 05:06:57.689239: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-11 05:07:13.464606: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the envi

The tokenizers are objects that are able to divide a python string into a list of tokens (words, punctuations, special tokens...) as a list of strings.

The special tokens are used for a particular reasons:
* *\<unk\>*: Replace an unknown word in the vocabulary by this default token
* *\<pad\>*: Virtual token used to as padding token so a batch of sentences can have a unique length
* *\<bos\>*: Token indicating the beggining of a sentence in the target sequence
* *\<eos\>*: Token indicating the end of a sentence in the target sequence

In [ ]:
# Original dataset, but there's a bug on Colab with it
# train, valid, _ = IWSLT2016(language_pair=('fr', 'en'))
# train, valid = list(train), list(valid)

# Another dataset, but it is too huge
# !wget https://www.statmt.org/wmt14/training-monolingual-europarl-v7/europarl-v7.en.gz
# !wget https://www.statmt.org/wmt14/training-monolingual-europarl-v7/europarl-v7.fr.gz
# !gunzip europarl-v7.en.gz
# !gunzip europarl-v7.fr.gz

# with open('europarl-v7.en', 'r') as my_file:
#     english = my_file.readlines()

# with open('europarl-v7.fr', 'r') as my_file:
#     french = my_file.readlines()

# dataset = [
#     (en, fr)
#     for en, fr in zip(english, french)
# ]
# print(f'\n{len(dataset):,} sentences.')

# dataset, _ = train_test_split(dataset, test_size=0.8, random_state=0)  # Remove 80% of the dataset (it would be huge otherwise)
# train, valid = train_test_split(dataset, test_size=0.2, random_state=0)  # Split between train and validation dataset

# Our current dataset
!wget http://www.manythings.org/anki/fra-eng.zip
!unzip fra-eng.zip


df = pd.read_csv('fra.txt', sep='\t', names=['english', 'french', 'attribution'])
train = [
    (en, fr) for en, fr in zip(df['english'], df['french'])
]
train, valid = train_test_split(train, test_size=0.1, random_state=0)
print(len(train))

en_tokenizer, fr_tokenizer = get_tokenizer('spacy', language='en'), get_tokenizer('spacy', language='fr')

SPECIALS = ['<unk>', '<pad>', '<bos>', '<eos>']

--2023-04-11 05:07:42--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7420323 (7.1M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.08M  21.7MB/s    in 0.3s    

2023-04-11 05:07:43 (21.7 MB/s) - ‘fra-eng.zip’ saved [7420323/7420323]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 
196177


/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "fr" could not be loaded, trying "fr_core_news_sm" instead
  warnings.warn(


## Datasets

Functions and classes to build the vocabularies and the torch datasets.
The vocabulary is an object able to transform a string token into the id (an int) of that token in the vocabulary.

In [ ]:
class TranslationDataset(Dataset):
    def __init__(
            self,
            dataset: list,
            en_vocab: Vocab,
            fr_vocab: Vocab,
            en_tokenizer,
            fr_tokenizer,
        ):
        super().__init__()

        self.dataset = dataset
        self.en_vocab = en_vocab
        self.fr_vocab = fr_vocab
        self.en_tokenizer = en_tokenizer
        self.fr_tokenizer = fr_tokenizer

    def __len__(self):
        """Return the number of examples in the dataset.
        """
        return len(self.dataset)

    def __getitem__(self, index: int) -> tuple:
        """Return a sample.

        Args
        ----
            index: Index of the sample.

        Output
        ------
            en_tokens: English tokens of the sample, as a LongTensor.
            fr_tokens: French tokens of the sample, as a LongTensor.
        """
        # Get the strings
        en_sentence, fr_sentence = self.dataset[index]

        # To list of words
        # We also add the beggining-of-sentence and end-of-sentence tokens
        en_tokens = ['<bos>'] + self.en_tokenizer(en_sentence) + ['<eos>']
        fr_tokens = ['<bos>'] + self.fr_tokenizer(fr_sentence) + ['<eos>']

        # To list of tokens
        en_tokens = self.en_vocab(en_tokens)  # list[int]
        fr_tokens = self.fr_vocab(fr_tokens)

        return torch.LongTensor(en_tokens), torch.LongTensor(fr_tokens)


def yield_tokens(dataset, tokenizer, lang):
    """Tokenize the whole dataset and yield the tokens.
    """
    assert lang in ('en', 'fr')
    sentence_idx = 0 if lang == 'en' else 1

    for sentences in dataset:
        sentence = sentences[sentence_idx]
        tokens = tokenizer(sentence)
        yield tokens


def build_vocab(dataset: list, en_tokenizer, fr_tokenizer, min_freq: int):
    """Return two vocabularies, one for each language.
    """
    en_vocab = build_vocab_from_iterator(
        yield_tokens(dataset, en_tokenizer, 'en'),
        min_freq=min_freq,
        specials=SPECIALS,
    )
    en_vocab.set_default_index(en_vocab['<unk>'])  # Default token for unknown words

    fr_vocab = build_vocab_from_iterator(
        yield_tokens(dataset, fr_tokenizer, 'fr'),
        min_freq=min_freq,
        specials=SPECIALS,
    )
    fr_vocab.set_default_index(fr_vocab['<unk>'])

    return en_vocab, fr_vocab


def preprocess(
        dataset: list,
        en_tokenizer,
        fr_tokenizer,
        max_words: int,
    ) -> list:
    """Preprocess the dataset.
    Remove samples where at least one of the sentences are too long.
    Those samples takes too much memory.
    Also remove the pending '\n' at the end of sentences.
    """
    filtered = []

    for en_s, fr_s in dataset:
        if len(en_tokenizer(en_s)) >= max_words or len(fr_tokenizer(fr_s)) >= max_words:
            continue

        en_s = en_s.replace('\n', '')
        fr_s = fr_s.replace('\n', '')

        filtered.append((en_s, fr_s))

    return filtered


def build_datasets(
        max_sequence_length: int,
        min_token_freq: int,
        en_tokenizer,
        fr_tokenizer,
        train: list,
        val: list,
    ) -> tuple:
    """Build the training, validation and testing datasets.
    It takes care of the vocabulary creation.

    Args
    ----
        - max_sequence_length: Maximum number of tokens in each sequences.
            Having big sequences increases dramatically the VRAM taken during training.
        - min_token_freq: Minimum number of occurences each token must have
            to be saved in the vocabulary. Reducing this number increases
            the vocabularies's size.
        - en_tokenizer: Tokenizer for the english sentences.
        - fr_tokenizer: Tokenizer for the french sentences.
        - train and val: List containing the pairs (english, french) sentences.


    Output
    ------
        - (train_dataset, val_dataset): Tuple of the two TranslationDataset objects.
    """
    datasets = [
        preprocess(samples, en_tokenizer, fr_tokenizer, max_sequence_length)
        for samples in [train, val]
    ]

    en_vocab, fr_vocab = build_vocab(datasets[0], en_tokenizer, fr_tokenizer, min_token_freq)

    datasets = [
        TranslationDataset(samples, en_vocab, fr_vocab, en_tokenizer, fr_tokenizer)
        for samples in datasets
    ]

    return datasets

In [ ]:
def generate_batch(data_batch: list, src_pad_idx: int, tgt_pad_idx: int) -> tuple:
    """Add padding to the given batch so that all
    the samples are of the same size.

    Args
    ----
        data_batch: List of samples.
            Each sample is a tuple of LongTensors of varying size.
        src_pad_idx: Source padding index value.
        tgt_pad_idx: Target padding index value.

    Output
    ------
        en_batch: Batch of tokens for the padded english sentences.
            Shape of [batch_size, max_en_len].
        fr_batch: Batch of tokens for the padded french sentences.
            Shape of [batch_size, max_fr_len].
    """
    en_batch, fr_batch = [], []
    for en_tokens, fr_tokens in data_batch:
        en_batch.append(en_tokens)
        fr_batch.append(fr_tokens)

    en_batch = pad_sequence(en_batch, padding_value=src_pad_idx, batch_first=True)
    fr_batch = pad_sequence(fr_batch, padding_value=tgt_pad_idx, batch_first=True)
    return en_batch, fr_batch

# Models architecture
This is where you have to code the architectures.

In a machine translation task, the model takes as input the whole
source sentence along with the current known tokens of the target,
and predict the next token in the target sequence.
This means that the target tokens are predicted in an autoregressive
manner, starting from the first token (right after the *\<bos\>* token) and producing tokens one by one until the last *\<eos\>* token.

Formally, we define $s = [s_1, ..., s_{N_s}]$ as the source sequence made of $N_s$ tokens.
We also define $t^i = [t_1, ..., t_i]$ as the target sequence at the beginning of the step $i$.

The output of the model parameterized by $\theta$ is:

$$
T_{i+1} = p(t_{i+1} | s, t^i ; \theta )
$$

Where $T_{i+1}$ is the distribution of the next token $t_{i+1}$.

The loss is simply a *cross entropy loss* over the whole steps, where each class is a token of the vocabulary.

![RNN schema for machinea translation](https://www.simplilearn.com/ice9/free_resources_article_thumb/machine-translation-model-with-encoder-decoder-rnn.jpg)

Note that in this image the english sentence is provided in reverse.

---

In pytorch, there is no dinstinction between an intermediate layer or a whole model having multiple layers in itself.
Every layers or models inherit from the `torch.nn.Module`.
This module needs to define the `__init__` method where you instanciate the layers,
and the `forward` method where you decide how the inputs and the layers of the module interact between them.
Thanks to the autograd computations of pytorch, you do not have
to implement any backward method!

A really important advice is to **always look at
the shape of your input and your output.**
From that, you can often guess how the layers should interact
with the inputs to produce the right output.
You can also easily detect if there's something wrong going on.

You are more than advised to use the `einops` library and the `torch.einsum` function. This will require less operations than 'classical' code, but note that it's a bit trickier to use.
This is a way of describing tensors manipulation with strings, bypassing the multiple tensor methods executed in the background.
You can find a nice presentation of `einops` [here](https://einops.rocks/1-einops-basics/).
A paper has just been released about einops [here](https://paperswithcode.com/paper/einops-clear-and-reliable-tensor).

**A great tutorial on pytorch can be found [here](https://stanford.edu/class/cs224n/materials/CS224N_PyTorch_Tutorial.html).**
Spending 3 hours on this tutorial is *no* waste of time.

## RNN models

### RNN
Here you have to implement a recurrent neural network. You will need to create a single RNN Layer, and a module allowing to stack these layers. Look up the pytorch documentation to figure out this module's operations and what is communicated from one layer to another.

The `RNNCell` layer produce one hidden state vector for each sentence in the batch
(useful for the output of the encoder), and also produce one embedding for each
token in each sentence (useful for the output of the decoder).

The `RNN` module is composed of a stack of `RNNCell`. Each token embeddings
coming out from a previous `RNNCell` is used as an input for the next `RNNCell` layer.

**Be careful !** Our `RNNCell` implementation is not exactly the same thing as
the PyTorch's `nn.RNNCell`. PyTorch implements only the operations for one token
(so you would need to loop through each tokens inside the `RNN` instead).
You are free to implement `RNN` and `RNNCell` the way you want, as long as it has the expected behaviour of a RNN.

The same thing apply for the `GRU` and `GRUCell`.


In [ ]:

class RNNCell(nn.Module):
    """A single RNN layer.

    Parameters
    ----------
        input_size: Size of each input token.
        hidden_size: Size of each RNN hidden state.
        dropout: Dropout rate.

    Important note: This layer does not exactly the same thing as nn.RNNCell does.
    PyTorch implementation is only doing one simple pass over one token for each batch.
    This implementation is taking the whole sequence of each batch and provide the
    final hidden state along with the embeddings of each token in each sequence.
    """
    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            dropout: float,
        ):
        super().__init__()

        self.Wih = nn.Linear(input_size, hidden_size)
        self.Whh = nn.Linear(hidden_size, hidden_size)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(p=dropout)
        self.hidden_size = hidden_size
        self.device = config["device"]


    def forward(self, x: torch.FloatTensor, h: torch.FloatTensor) -> tuple:
        """Go through all the sequence in x, iteratively updating
        the hidden state h.

        Args
        ----
            x: Input sequence.
                Shape of [batch_size, seq_len, input_size].
            h: Initial hidden state.
                Shape of [batch_size, hidden_size].

        Output
        ------
            y: Token embeddings.
                Shape of [batch_size, seq_len, hidden_size].
            h: Last hidden state.
                Shape of [batch_size, hidden_size].
        """
        batch_size, seq_len, _ = x.size()

        y = torch.empty(batch_size, seq_len, self.hidden_size).to(self.device)
        for i in range(seq_len):
            ih = self.Wih(x[:, i])
            h = self.Whh(h)
            h = self.tanh(h + ih)
            y[:, i] = h

        h = self.dropout(h)
        y = self.dropout(y)
        return y, h



class RNN(nn.Module):
    """Implementation of an RNN based
    on https://pytorch.org/docs/stable/generated/torch.nn.RNN.html.

    Parameters
    ----------
        input_size: Size of each input token.
        hidden_size: Size of each RNN hidden state.
        num_layers: Number of layers (RNNCell or GRUCell).
        dropout: Dropout rate.
        model_type: Either 'RNN' or 'GRU', to select which model we want.
            This parameter can be removed if you decide to use the module GRU.
            Indeed, GRU should have exactly the same code as this module,
            but with GRUCell instead of RNNCell. We let the freedom for you
            to decide at which level you want to specialise the modules (either
            in TranslationRNN by creating a GRU or a RNN, or in RNN
            by creating a GRUCell or a RNNCell).
    """
    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            num_layers: int,
            dropout: float,
            model_type: str,
        ):
        super().__init__()

        self.num_layers = num_layers


        self.hidden_size = hidden_size
        self.device = config["device"]


        self.model = nn.ModuleList([
            RNNCell(
                input_size if l == 0 else hidden_size,
                hidden_size,
                dropout if l != num_layers - 1 else 0
            ) if model_type == 'RNN' else
            GRUCell(
                input_size if l == 0 else hidden_size,
                hidden_size,
                dropout if l != num_layers - 1 else 0
            ) for l in range(num_layers)
        ])

    def forward(self, x: torch.FloatTensor, h: torch.FloatTensor=None) -> tuple:
        """Pass the input sequence through all the RNN cells.
        Returns the output and the final hidden state of each RNN layer

        Args
        ----
            x: Input sequence.
                Shape of [batch_size, seq_len, input_size].
            h: Hidden state for each RNN layer.
                Can be None, in which case an initial hidden state is created.
                Shape of [batch_size, n_layers, hidden_size].

        Output
        ------
            y: Output embeddings for each token after the RNN layers.
                Shape of [batch_size, seq_len, hidden_size].
            h: Final hidden state.
                Shape of [batch_size, n_layers, hidden_size].
        """
        batch_size,_,_ = x.size()
        output_emb = x
        final_hidden_state = torch.empty(batch_size, self.num_layers, self.hidden_size).to(self.device)

        if h == None:
            h = torch.zeros(batch_size, self.num_layers, self.hidden_size).to(self.device)

        for i, Rnncell in enumerate(self.model):
            hh = h[:, i]
            output_emb, hh = Rnncell(output_emb, hh)

            final_hidden_state[:, i] = hh
        return output_emb, final_hidden_state

### GRU
Here you have to implement a GRU-RNN. This architecture is close to the Vanilla RNN but perform different operations. Look up the pytorch documentation to figure out the differences.

In [ ]:
class GRU(nn.Module):
    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            num_layers: int,
            dropout: float,
        ):
        super().__init__()

        self.h_size = hidden_size
        self.n_layers = num_layers
        self.device = config["device"]

        # Create GRU layers
        self.gru_layers = nn.ModuleList([GRUCell(
            input_size if n == 0 else hidden_size,
            hidden_size,
            dropout if n != (num_layers-1) else 0) for n in range(num_layers)])

    def forward(self, x: torch.FloatTensor, h: torch.FloatTensor=None) -> tuple:
        batch_size, _, _ = x.size()

        out_seq = x

        final_h = torch.empty(batch_size, self.n_layers, self.h_size).to(self.device)

        if h == None:
            h = torch.zeros(batch_size, self.n_layers, self.h_size).to(self.device)

        for i, cell in enumerate(self.gru_layers):
            h_layer = h[:, i]
            out_seq, h_layer = cell(out_seq, h_layer)
            final_h[:, i] = h_layer
        return out_seq, final_h


class GRUCell(nn.Module):
    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            dropout: float,
        ):
        super().__init__()

        self.gate_size = 3

        # Define the weights for the input and hidden states
        self.W_input = nn.Linear(input_size, hidden_size * self.gate_size)
        self.W_hidden = nn.Linear(hidden_size, hidden_size * self.gate_size)

        self.device = config["device"]
        self.h_size = hidden_size
        self.dropout_layer = nn.Dropout(p=dropout)

        std = 1.0 / np.sqrt(self.h_size)

        # Initialize the weights
        for weight in self.parameters():
            weight.data.uniform_(-std, std)

    def forward(self, x: torch.FloatTensor, h: torch.FloatTensor) -> tuple:
        batch_size, seq_len, _ = x.size()

        out_emb = torch.empty(batch_size, seq_len, self.h_size).to(self.device)

        final_h = h

        for token in range(seq_len):
            x_gate = self.W_input(x[:, token])
            h_gate = self.W_hidden(final_h)

            # Split input and hidden states into gates
            i_r, i_u, i_n = x_gate.chunk(self.gate_size, 1)
            h_r, h_u, h_n = h_gate.chunk(self.gate_size, 1)

            # Compute gate activations
            reset_gate = torch.sigmoid(i_r + h_r)
            update_gate = torch.sigmoid(i_u + h_u)
            new_candidate_state = torch.tanh(i_n + (reset_gate * h_n))

            final_h = update_gate * final_h + (1 - update_gate) * new_candidate_state

            out_emb[:, token] = final_h

        out_emb = self.dropout_layer(out_emb)
        final_h = self.dropout_layer(final_h)

        return out_emb, final_h

### Translation RNN

This module instanciates a vanilla RNN or a GRU-RNN and performs the translation task. You have to:
* Encode the source and target sequence
* Pass the final hidden state of the encoder to the decoder (one for each layer)
* Decode the hidden state into the target sequence

We use teacher forcing for training, meaning that when the next token is predicted, that prediction is based on the previous true target tokens.

In [ ]:

class TranslationRNN(nn.Module):
    """Basic RNN encoder and decoder for a translation task.
    It can run as a vanilla RNN or a GRU-RNN.

    Parameters
    ----------
        n_tokens_src: Number of tokens in the source vocabulary.
        n_tokens_tgt: Number of tokens in the target vocabulary.
        dim_embedding: Dimension size of the word embeddings (for both language).
        dim_hidden: Dimension size of the hidden layers in the RNNs
            (for both the encoder and the decoder).
        n_layers: Number of layers in the RNNs.
        dropout: Dropout rate.
        src_pad_idx: Source padding index value.
        tgt_pad_idx: Target padding index value.
        model_type: Either 'RNN' or 'GRU', to select which model we want.
    """

    def __init__(
            self,
            n_tokens_src: int,
            n_tokens_tgt: int,
            dim_embedding: int,
            dim_hidden: int,
            n_layers: int,
            dropout: float,
            src_pad_idx: int,
            tgt_pad_idx: int,
            model_type: str,
        ):
        super().__init__()

        # TODO
        self.device = config['device']
        self.dim_hidden = dim_hidden
        self.n_layers = n_layers

        self.embedding_src = nn.Embedding(num_embeddings=n_tokens_src, embedding_dim=dim_embedding, padding_idx=src_pad_idx)
        self.embedding_tgt = nn.Embedding(num_embeddings=n_tokens_tgt, embedding_dim=dim_embedding, padding_idx=tgt_pad_idx)

        self.encoder = None
        self.decoder = None

        if model_type == 'RNN':
            self.encoder = RNN(input_size=dim_embedding,
                            hidden_size=dim_hidden,
                            num_layers=n_layers,
                            dropout=dropout,
                            model_type=model_type)
            self.decoder = RNN(input_size=dim_embedding,
                            hidden_size=dim_hidden,
                            num_layers=n_layers,
                            dropout=dropout,
                            model_type=model_type)

        elif model_type == 'GRU':
            self.encoder = GRU(input_size=dim_embedding,
                            hidden_size=dim_hidden,
                            num_layers=n_layers,
                            dropout=dropout)
            self.decoder = GRU(input_size=dim_embedding,
                            hidden_size=dim_hidden,
                            num_layers=n_layers,
                            dropout=dropout)

        self.layer_norm = nn.LayerNorm(dim_hidden)

        self.mlp = nn.Sequential(
            nn.Linear(dim_hidden, 128),
            nn.LeakyReLU(),
            nn.LayerNorm(128),
            nn.Linear(128, 128),
            nn.LeakyReLU(),
            nn.LayerNorm(128),
            nn.Linear(128, 128),
            nn.LeakyReLU(),
            nn.LayerNorm(128),
            nn.Linear(128, n_tokens_tgt)
        )


    def forward(
        self,
        source: torch.LongTensor,
        target: torch.LongTensor
    ) -> torch.FloatTensor:
        """Predict the target tokens logites based on the source tokens.

        Args
        ----
            source: Batch of source sentences.
                Shape of [batch_size, src_seq_len].
            target: Batch of target sentences.
                Shape of [batch_size, tgt_seq_len].

        Output
        ------
            y: Distributions over the next token for all tokens in each sentences.
                Those need to be the logits only, do not apply a softmax because
                it will be done in the loss computation for numerical stability.
                See https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html for more informations.
                Shape of [batch_size, tgt_seq_len, n_tokens_tgt].
        """


        x_source = self.embedding_src(source)
        _, h_enc = self.encoder(x_source)
        x_target = self.embedding_tgt(target)
        h_enc = self.layer_norm(h_enc)
        x_dec, _ = self.decoder(x_target, h_enc)
        y = self.mlp(x_dec)

        return y

## Transformer model
Here you have to code the Transformer architecture.
It is divided in three parts:
* Attention layers
* Encoder and decoder layers
* Main layers (gather the encoder and decoder layers)

The [illustrated transformer](https://jalammar.github.io/illustrated-transformer/) blog can help you
understanding how the architecture works.
Once this is done, you can use [the annontated transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html) to have an idea of how to code this architecture.
We encourage you to use `torch.einsum` and the `einops` library as much as you can. It will make your code simpler.

---
**Implementation order**

To help you with the implementation, we advise you following this order:
* Implement `TranslationTransformer` and use `nn.Transformer` instead of `Transformer`
* Implement `Transformer` and use `nn.TransformerDecoder` and `nn.TransformerEnocder`
* Implement the `TransformerDecoder` and `TransformerEncoder` and use `nn.MultiHeadAttention`
* Implement `MultiHeadAttention`

Do not forget to add `batch_first=True` when necessary in the `nn` modules.

### Attention layers
We use a `MultiHeadAttention` module, that is able to perform self-attention aswell as cross-attention (depending on what you give as queries, keys and values).

**Attention**


It takes the multiheaded queries, keys and values as input.
It computes the attention between the queries and the keys and return the attended values.

The implementation of this function can greatly be improved with *einsums*.

**MultiheadAttention**

Computes the multihead queries, keys and values and feed them to the `attention` function.
You also need to merge the key padding mask and the attention mask into one mask.

The implementation of this module can greatly be improved with *einops.rearrange*.

In [ ]:
from einops.layers.torch import Rearrange

import math
def attention(
        q: torch.FloatTensor,
        k: torch.FloatTensor,
        v: torch.FloatTensor,
        mask: torch.BoolTensor=None,
        dropout: nn.Dropout=None,
    ) -> tuple:
    """
    Computes multihead scaled dot-product attention from the
    projected queries, keys and values.

    Args:
        q: Batch of queries.
            Shape of [batch_size, seq_len_1, n_heads, dim_model].
        k: Batch of keys.
            Shape of [batch_size, seq_len_2, n_heads, dim_model].
        v: Batch of values.
            Shape of [batch_size, seq_len_2, n_heads, dim_model].
        mask: Prevent tokens to attend to some other tokens (for padding or autoregressive attention).
            Attention is prevented where the mask is True.
            Shape of [batch_size, 1, seq_len_1, seq_len_2],
            or broadcastable to that shape.
        dropout: Dropout layer to use.

    Returns:
        y: Multihead scaled dot-attention between the queries, keys and values.
            Shape of [batch_size, seq_len_1, n_heads, dim_model].
        attn: Computed attention between the keys and the queries.
            Shape of [batch_size, n_heads, seq_len_1, seq_len_2].
    """
    # Calculate the dot-product of q and k for each head
    # Shape of [batch_size, n_heads, seq_len_1, seq_len_2]
    #scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(q.size(-1))
    scores = torch.einsum("bqhd,bkhd->bhqk", [q, k]) / (d_k ** 0.5)
    # Apply mask to the score#
    print('q',q.shape)
    print('k',k.shape)
    print('score',scores.shape)
    print('mask',mask.shape)
    print(q.size(1))
    #mask = mask.expand(-1, q.size(1), -1, -1, -1)
    mask = mask.expand(q.size(0), 1, q.size(1), v.size(1))
    print('mask',mask.shape)
    scores.masked_fill_(mask, -1e9)

    # Apply softmax to the score
    # Shape of [batch_size, n_heads, seq_len_1, seq_len_2]
    attn_weights = nn.softmax(scores, dim=-1)

    # Apply dropout
    if dropout is not None:
        attn_weights = dropout(attn_weights)

    # Calculate the weighted sum of values
    # Shape of [batch_size, seq_len_1, n_heads, dim_model]
    y = torch.matmul(attn_weights, v)

    # Transpose the dimensions for compatibility with nn.MultiheadAttention
    # Shape of [batch_size, seq_len_1, n_heads, dim_model]
    y = y.transpose(1, 2)

    # Transpose the attention weights dimensions
    # Shape of [batch_size, n_heads, seq_len_1, seq_len_2]
    attn_weights = attn_weights.transpose(1, 2)

    return y, attn_weights



import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiheadAttention(nn.Module):
    def __init__(self, dim, n_heads, dropout):
        super().__init__()
        assert dim % n_heads == 0

        self.dim = dim
        self.n_heads = n_heads
        self.head_dim = dim // n_heads
        self.dropout = dropout

        self.q_proj = nn.Linear(dim, dim)
        self.k_proj = nn.Linear(dim, dim)
        self.v_proj = nn.Linear(dim, dim)
        self.out_proj = nn.Linear(dim, dim)

        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, q, k, v, key_padding_mask=None, attn_mask=None):
        batch_size, seq_len_1, dim_model = q.size()
        seq_len_2 = k.size(1)

        q = self.q_proj(q).view(batch_size, seq_len_1, self.n_heads, self.head_dim)
        k = self.k_proj(k).view(batch_size, seq_len_2, self.n_heads, self.head_dim)
        v = self.v_proj(v).view(batch_size, seq_len_2, self.n_heads, self.head_dim)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        if attn_mask is not None:
            attn_mask = attn_mask.unsqueeze(0).unsqueeze(1)

        if key_padding_mask is not None:
            key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)

        if attn_mask is None:
            mask = key_padding_mask
        else:
            mask = attn_mask if key_padding_mask is None else (key_padding_mask | attn_mask)

        attn_weights = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            attn_weights = attn_weights.masked_fill(mask, float("-inf"))

        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_weights = self.dropout_layer(attn_weights)

        y = torch.matmul(attn_weights, v)
        y = y.transpose(1, 2).contiguous().view(batch_size, seq_len_1, dim_model)
        y = self.out_proj(y)

        return y


In [ ]:
from numpy.core.fromnumeric import shape
from einops.layers.torch import Rearrange

def attention(
    q: torch.FloatTensor,
    k: torch.FloatTensor,
    v: torch.FloatTensor,
    mask: torch.BoolTensor = None,
    dropout: nn.Dropout = None,
) -> tuple:

    d_k = q.size(-1)
    print("q",q.shape)
    print("k",k.shape)
    print("v",v.shape)
    scores = torch.einsum("bqhd,bkhd->bhqk", [q, k]) / (d_k ** 0.5)
    print("scores",scores.shape)
    if mask is not None:
        print("mask",mask.shape)
        mask = mask.unsqueeze(1).unsqueeze(2).unsqueeze(2).expand(-1, 4, 180, 180)
        print("mask1",mask.shape)
        scores = scores.masked_fill(mask, float("-inf"))

    attn = torch.softmax(scores, dim=-1)

    if dropout is not None:
        attn = dropout(attn)

    y = torch.einsum("bhqk,bkhd->bqhd", [attn, v])

    return y, attn





class MultiheadAttention(nn.Module):
    def __init__(
            self,
            dim: int,
            n_heads: int,
            dropout: float,
        ):
        super().__init__()

        assert dim % n_heads == 0

        self.dim = dim
        self.n_heads = n_heads
        self.head_dim = dim // n_heads

        self.qkv_proj = nn.Linear(dim, dim * 3, bias=False)
        self.out_proj = nn.Linear(dim, dim, bias=False)
        self.dropout = nn.Dropout(dropout)

        self.rearrange_qkv = Rearrange("b s (h d qkv) -> qkv b s h d", h=n_heads, qkv=3)
        self.rearrange_out = Rearrange("b s h d -> b s (h d)")

    def forward(
            self,
            q: torch.FloatTensor,
            k: torch.FloatTensor,
            v: torch.FloatTensor,
            key_padding_mask: torch.BoolTensor = None,
            attn_mask: torch.BoolTensor = None,
        ) -> torch.FloatTensor:

        b, _, _ = q.size()

        qkv = self.qkv_proj(torch.cat([q, k, v], dim=1))
        q, k, v = self.rearrange_qkv(qkv).unbind(0)
        if key_padding_mask is not None:
            key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
            print("key_padding_mask size after unsqueeze:", key_padding_mask.size())


        if attn_mask is not None:
            attn_mask = attn_mask.unsqueeze(0).unsqueeze(1)


        if attn_mask is None:
            mask = key_padding_mask
        else:
            key_padding_mask_broadcasted = key_padding_mask.expand(-1, -1, attn_mask.size(2), -1)
            mask = key_padding_mask_broadcasted | attn_mask

        y, _ = attention(q, k, v, mask, self.dropout)

        y = self.rearrange_out(y)
        y = self.out_proj(y)

        return y

### Encoder and decoder layers

**TranformerEncoder**

Apply self-attention layers onto the source tokens.
It only needs the source key padding mask.


**TranformerDecoder**

Apply masked self-attention layers to the target tokens and cross-attention
layers between the source and the target tokens.
It needs the source and target key padding masks, and the target attention mask.

In [ ]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model: int, d_ff: int, nhead: int, dropout: float):
        super().__init__()

        self.self_attn = MultiheadAttention(d_model, nhead, dropout)
        self.cross_attn = MultiheadAttention(d_model, nhead, dropout)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

    def forward(self, src, tgt, tgt_mask_attn, src_key_padding_mask, tgt_key_padding_mask):
        attn_output = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask_attn, key_padding_mask=tgt_key_padding_mask)
        tgt = self.norm1(tgt + attn_output)
        attn_output = self.cross_attn(tgt, src, src, key_padding_mask=src_key_padding_mask)
        tgt = self.norm2(tgt + attn_output)
        ffn_output = self.ffn(tgt)
        tgt = self.norm3(tgt + ffn_output)

        return tgt


class TransformerDecoder(nn.Module):
    def __init__(self, d_model: int, d_ff: int, num_decoder_layer: int, nhead: int, dropout: float):
        super().__init__()

        self.layers = nn.ModuleList([TransformerDecoderLayer(d_model, d_ff, nhead, dropout) for _ in range(num_decoder_layer)])

    def forward(self, src, tgt, tgt_mask_attn, src_key_padding_mask, tgt_key_padding_mask):
        for layer in self.layers:
            tgt = layer(src, tgt, tgt_mask_attn, src_key_padding_mask, tgt_key_padding_mask)

        return tgt


class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model: int, d_ff: int, nhead: int, dropout: float):
        super().__init__()

        self.self_attn = MultiheadAttention(d_model, nhead, dropout)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, src, key_padding_mask):
        attn_output = self.self_attn(src, src, src, key_padding_mask=key_padding_mask)
        src = self.norm1(src + attn_output)
        ffn_output = self.ffn(src)
        src = self.norm2(src + ffn_output)

        return src


class TransformerEncoder(nn.Module):
    def __init__(self, d_model: int, dim_feedforward: int, num_encoder_layers: int, nheads: int, dropout: float):
        super().__init__()

        self.layers = nn.ModuleList([TransformerEncoderLayer(d_model, dim_feedforward, nheads, dropout) for _ in range(num_encoder_layers)])

    def forward(self, src, key_padding_mask):
        for layer in self.layers:
            src = layer(src, key_padding_mask)

        return src

### Main layers
This section gather the `Transformer` and the `TranslationTransformer` modules.

**Transformer**


The classical transformer architecture.
It takes the source and target tokens embeddings and
do the forward pass through the encoder and decoder.

**Translation Transformer**

Compute the source and target tokens embeddings, and apply a final head to produce next token logits.
The output must not be the softmax but just the logits, because we use the `nn.CrossEntropyLoss`.

It also creates the *src_key_padding_mask*, the *tgt_key_padding_mask* and the *tgt_mask_attn*.

In [ ]:
class Transformer(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout):
        super().__init__()

        self.encoder = TransformerEncoder(d_model, dim_feedforward, num_encoder_layers, nhead, dropout)
        self.decoder = TransformerDecoder(d_model, dim_feedforward, num_decoder_layers, nhead, dropout)

    def forward(self, src, tgt, tgt_mask_attn, src_key_padding_mask, tgt_key_padding_mask):
        memory = self.encoder(src, src_key_padding_mask)
        output = self.decoder(memory, tgt, tgt_mask_attn, src_key_padding_mask, tgt_key_padding_mask)

        return output


class TranslationTransformer(nn.Module):
    def __init__(self, n_tokens_src, n_tokens_tgt, n_heads, dim_embedding, dim_hidden, n_layers, dropout, src_pad_idx, tgt_pad_idx):
        super().__init__()

        self.src_embedding = nn.Embedding(n_tokens_src, dim_embedding)
        self.tgt_embedding = nn.Embedding(n_tokens_tgt, dim_embedding)
        self.position_encoding = nn.Embedding(dim_embedding, dim_embedding)
        self.transformer = Transformer(dim_embedding, n_heads, n_layers, n_layers, dim_hidden, dropout)
        self.fc_out = nn.Linear(dim_embedding, n_tokens_tgt)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx
        self.tgt_pad_idx = tgt_pad_idx

    def make_src_key_padding_mask(self, source):
        return source == self.src_pad_idx

    def make_tgt_key_padding_mask(self, target):
        return target == self.tgt_pad_idx

    def make_tgt_mask(self, tgt):
        tgt_seq_len = tgt.shape[1]
        tgt_mask = torch.triu(torch.ones((tgt_seq_len, tgt_seq_len), device=tgt.device), diagonal=1).bool()
        return tgt_mask

    def forward(self, source, target):
        src_seq_len, tgt_seq_len = source.shape[1], target.shape[1]
        src_positions = torch.arange(0, src_seq_len).unsqueeze(0).repeat(source.shape[0], 1).to(source.device)
        tgt_positions = torch.arange(0, tgt_seq_len).unsqueeze(0).repeat(target.shape[0], 1).to(target.device)

        src = self.dropout(self.src_embedding(source) + self.position_encoding(src_positions))
        tgt = self.dropout(self.tgt_embedding(target) + self.position_encoding(tgt_positions))

        src_key_padding_mask = self.make_src_key_padding_mask(source)
        tgt_key_padding_mask = self.make_tgt_key_padding_mask(target)
        tgt_mask_attn = self.make_tgt_mask(target)

        transformer_output = self.transformer(src, tgt, tgt_mask_attn, src_key_padding_mask, tgt_key_padding_mask)

        logits = self.fc_out(transformer_output)

        return logits

# Greedy search

Here you have to implement a geedy search to generate a target translation from a trained model and an input source string.
The next token will simply be the most probable one.

In [ ]:
def greedy_search(model, source, src_vocab, tgt_vocab, src_tokenizer, device, max_sentence_length):
    model.eval()
    model.to(device)

    tokens = src_tokenizer(source)
    src_indexes = [src_vocab.stoi(token) for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    # Initialize target tensor with start of sequence token
    tgt_indexes = [tgt_vocab.stoi(tgt_vocab.SOS_TOKEN)]
    eos_token_idx = tgt_vocab.stoi(tgt_vocab.EOS_TOKEN)

    for _ in range(max_sentence_length):
        tgt_tensor = torch.LongTensor(tgt_indexes).unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model(src_tensor, tgt_tensor)

        # Get the highest probability token and its index
        _, next_token_idx = logits[:, -1, :].max(dim=-1)

        next_token_idx = next_token_idx.item()
        tgt_indexes.append(next_token_idx)

        if next_token_idx == eos_token_idx:
            break

    tgt_tokens = [tgt_vocab.itos(idx) for idx in tgt_indexes]

    sentence = ' '.join(tgt_tokens[1:])

    return sentence

# Beam search
Beam search is a smarter way of producing a sequence of tokens from
an autoregressive model than just using a greedy search.

The greedy search always choose the most probable token as the unique
and only next target token, and repeat this processus until the *\<eos\>* token is predicted.

Instead, the beam search selects the k-most probable tokens at each step.
From those k tokens, the current sequence is duplicated k times and the k tokens are appended to the k sequences to produce new k sequences.

*You don't have to understand this code, but understanding this code once the TP is over could improve your torch tensors skills.*

---

**More explanations**

Since it is done at each step, the number of sequences grows exponentially (k sequences after the first step, k² sequences after the second...).
In order to keep the number of sequences low, we remove sequences except the top-s most likely sequences.
To do that, we keep track of the likelihood of each sequence.

Formally, we define $s = [s_1, ..., s_{N_s}]$ as the source sequence made of $N_s$ tokens.
We also define $t^i = [t_1, ..., t_i]$ as the target sequence at the beginning of the step $i$.

The output of the model parameterized by $\theta$ is:

$$
T_{i+1} = p(t_{i+1} | s, t^i ; \theta )
$$

Where $T_{i+1}$ is the distribution of the next token $t_{i+1}$.

Then, we define the likelihood of a target sentence $t = [t_1, ..., t_{N_t}]$ as:

$$
L(t) = \prod_{i=1}^{N_t - 1} p(t_{i+1} | s, t_{i}; \theta )
$$

Pseudocode of the beam search:
```
source: [N_s source tokens]  # Shape of [total_source_tokens]
target: [1, <bos> token]  # Shape of [n_sentences, current_target_tokens]
target_prob: [1]  # Shape of [n_sentences]
# We use `n_sentences` as the batch_size dimension

while current_target_tokens <= max_target_length:
    source = repeat(source, n_sentences)  # Shape of [n_sentences, total_source_tokens]
    predicted = model(source, target)[:, -1]  # Predict the next token distributions of all the n_sentences
    tokens_idx, tokens_prob = topk(predicted, k)

    # Append the `n_sentences * k` tokens to the `n_sentences` sentences
    target = repeat(target, k)  # Shape of [n_sentences * k, current_target_tokens]
    target = append_tokens(target, tokens_idx)  # Shape of [n_sentences * k, current_target_tokens + 1]

    # Update the sentences probabilities
    target_prob = repeat(target_prob, k)  # Shape of [n_sentences * k]
    target_prob *= tokens_prob

    if n_sentences * k >= max_sentences:
        target, target_prob = topk_prob(target, target_prob, k=max_sentences)
    else:
        n_sentences *= k

    current_target_tokens += 1
```

In [ ]:
def beautify(sentence: str) -> str:
    """Removes useless spaces.
    """
    punc = {'.', ',', ';'}
    for p in punc:
        sentence = sentence.replace(f' {p}', p)

    links = {'-', "'"}
    for l in links:
        sentence = sentence.replace(f'{l} ', l)
        sentence = sentence.replace(f' {l}', l)

    return sentence


In [ ]:
def indices_terminated(
        target: torch.FloatTensor,
        eos_token: int
    ) -> tuple:
    """Split the target sentences between the terminated and the non-terminated
    sentence. Return the indices of those two groups.

    Args
    ----
        target: The sentences.
            Shape of [batch_size, n_tokens].
        eos_token: Value of the End-of-Sentence token.

    Output
    ------
        terminated: Indices of the terminated sentences (who's got the eos_token).
            Shape of [n_terminated, ].
        non-terminated: Indices of the unfinished sentences.
            Shape of [batch_size-n_terminated, ].
    """
    terminated = [i for i, t in enumerate(target) if eos_token in t]
    non_terminated = [i for i, t in enumerate(target) if eos_token not in t]
    return torch.LongTensor(terminated), torch.LongTensor(non_terminated)


def append_beams(
        target: torch.FloatTensor,
        beams: torch.FloatTensor
    ) -> torch.FloatTensor:
    """Add the beam tokens to the current sentences.
    Duplicate the sentences so one token is added per beam per batch.

    Args
    ----
        target: Batch of unfinished sentences.
            Shape of [batch_size, n_tokens].
        beams: Batch of beams for each sentences.
            Shape of [batch_size, n_beams].

    Output
    ------
        target: Batch of sentences with one beam per sentence.
            Shape of [batch_size * n_beams, n_tokens+1].
    """
    batch_size, n_beams = beams.shape
    n_tokens = target.shape[1]

    target = einops.repeat(target, 'b t -> b c t', c=n_beams)  # [batch_size, n_beams, n_tokens]
    beams = beams.unsqueeze(dim=2)  # [batch_size, n_beams, 1]

    target = torch.cat((target, beams), dim=2)  # [batch_size, n_beams, n_tokens+1]
    target = target.view(batch_size*n_beams, n_tokens+1)  # [batch_size * n_beams, n_tokens+1]
    return target


def beam_search(
        model: nn.Module,
        source: str,
        src_vocab: Vocab,
        tgt_vocab: Vocab,
        src_tokenizer,
        device: str,
        beam_width: int,
        max_target: int,
        max_sentence_length: int,
    ) -> list:
    """Do a beam search to produce probable translations.

    Args
    ----
        model: The translation model. Assumes it produces linear score (before softmax).
        source: The sentence to translate.
        src_vocab: The source vocabulary.
        tgt_vocab: The target vocabulary.
        device: Device to which we make the inference.
        beam_width: Number of top-k tokens we keep at each stage.
        max_target: Maximum number of target sentences we keep at the end of each stage.
        max_sentence_length: Maximum number of tokens for the translated sentence.

    Output
    ------
        sentences: List of sentences orderer by their likelihood.
    """
    src_tokens = ['<bos>'] + src_tokenizer(source) + ['<eos>']
    src_tokens = src_vocab(src_tokens)

    tgt_tokens = ['<bos>']
    tgt_tokens = tgt_vocab(tgt_tokens)

    # To tensor and add unitary batch dimension
    src_tokens = torch.LongTensor(src_tokens).to(device)
    tgt_tokens = torch.LongTensor(tgt_tokens).unsqueeze(dim=0).to(device)
    target_probs = torch.FloatTensor([1]).to(device)
    model.to(device)

    EOS_IDX = tgt_vocab['<eos>']
    with torch.no_grad():
        while tgt_tokens.shape[1] < max_sentence_length:
            batch_size, n_tokens = tgt_tokens.shape

            # Get next beams
            src = einops.repeat(src_tokens, 't -> b t', b=tgt_tokens.shape[0])
            predicted = model.forward(src, tgt_tokens)
            predicted = torch.softmax(predicted, dim=-1)
            probs, predicted = predicted[:, -1].topk(k=beam_width, dim=-1)

            # Separe between terminated sentences and the others
            idx_terminated, idx_not_terminated = indices_terminated(tgt_tokens, EOS_IDX)
            idx_terminated, idx_not_terminated = idx_terminated.to(device), idx_not_terminated.to(device)

            tgt_terminated = torch.index_select(tgt_tokens, dim=0, index=idx_terminated)
            tgt_probs_terminated = torch.index_select(target_probs, dim=0, index=idx_terminated)

            filter_t = lambda t: torch.index_select(t, dim=0, index=idx_not_terminated)
            tgt_others = filter_t(tgt_tokens)
            tgt_probs_others = filter_t(target_probs)
            predicted = filter_t(predicted)
            probs = filter_t(probs)

            # Add the top tokens to the previous target sentences
            tgt_others = append_beams(tgt_others, predicted)

            # Add padding to terminated target
            padd = torch.zeros((len(tgt_terminated), 1), dtype=torch.long, device=device)
            tgt_terminated = torch.cat(
                (tgt_terminated, padd),
                dim=1
            )

            # Update each target sentence probabilities
            tgt_probs_others = torch.repeat_interleave(tgt_probs_others, beam_width)
            tgt_probs_others *= probs.flatten()
            tgt_probs_terminated *= 0.999  # Penalize short sequences overtime

            # Group up the terminated and the others
            target_probs = torch.cat(
                (tgt_probs_others, tgt_probs_terminated),
                dim=0
            )
            tgt_tokens = torch.cat(
                (tgt_others, tgt_terminated),
                dim=0
            )

            # Keep only the top `max_target` target sentences
            if target_probs.shape[0] <= max_target:
                continue

            target_probs, indices = target_probs.topk(k=max_target, dim=0)
            tgt_tokens = torch.index_select(tgt_tokens, dim=0, index=indices)

    sentences = []
    for tgt_sentence in tgt_tokens:
        tgt_sentence = list(tgt_sentence)[1:]  # Remove <bos> token
        tgt_sentence = list(takewhile(lambda t: t != EOS_IDX, tgt_sentence))
        tgt_sentence = ' '.join(tgt_vocab.lookup_tokens(tgt_sentence))
        sentences.append(tgt_sentence)

    sentences = [beautify(s) for s in sentences]

    # Join the sentences with their likelihood
    sentences = [(s, p.item()) for s, p in zip(sentences, target_probs)]
    # Sort the sentences by their likelihood
    sentences = [(s, p) for s, p in sorted(sentences, key=lambda k: k[1], reverse=True)]

    return sentences

# Training loop
This is a basic training loop code. It takes a big configuration dictionnary to avoid never ending arguments in the functions.
We use [Weights and Biases](https://wandb.ai/) to log the trainings.
It logs every training informations and model performances in the cloud.
You have to create an account to use it. Every accounts are free for individuals or research teams.

In [ ]:
def print_logs(dataset_type: str, logs: dict):
    """Print the logs.

    Args
    ----
        dataset_type: Either "Train", "Eval", "Test" type.
        logs: Containing the metric's name and value.
    """
    desc = [
        f'{name}: {value:.2f}'
        for name, value in logs.items()
    ]
    desc = '\t'.join(desc)
    desc = f'{dataset_type} -\t' + desc
    desc = desc.expandtabs(5)
    print(desc)


def topk_accuracy(
        real_tokens: torch.FloatTensor,
        probs_tokens: torch.FloatTensor,
        k: int,
        tgt_pad_idx: int,
    ) -> torch.FloatTensor:
    """Compute the top-k accuracy.
    We ignore the PAD tokens.

    Args
    ----
        real_tokens: Real tokens of the target sentence.
            Shape of [batch_size * n_tokens].
        probs_tokens: Tokens probability predicted by the model.
            Shape of [batch_size * n_tokens, n_target_vocabulary].
        k: Top-k accuracy threshold.
        src_pad_idx: Source padding index value.

    Output
    ------
        acc: Scalar top-k accuracy value.
    """
    total = (real_tokens != tgt_pad_idx).sum()

    _, pred_tokens = probs_tokens.topk(k=k, dim=-1)            # [batch_size * n_tokens, k]
    real_tokens = einops.repeat(real_tokens, 'b -> b k', k=k)  # [batch_size * n_tokens, k]

    good = (pred_tokens == real_tokens) & (real_tokens != tgt_pad_idx)
    acc = good.sum() / total
    return acc


def loss_batch(
        model: nn.Module,
        source: torch.LongTensor,
        target: torch.LongTensor,
        config: dict,
    )-> dict:
    """Compute the metrics associated with this batch.
    The metrics are:
        - loss
        - top-1 accuracy
        - top-5 accuracy
        - top-10 accuracy

    Args
    ----
        model: The model to train.
        source: Batch of source tokens.
            Shape of [batch_size, n_src_tokens].
        target: Batch of target tokens.
            Shape of [batch_size, n_tgt_tokens].
        config: Additional parameters.

    Output
    ------
        metrics: Dictionnary containing evaluated metrics on this batch.
    """
    device = config['device']
    loss_fn = config['loss'].to(device)
    metrics = dict()

    source, target = source.to(device), target.to(device)
    target_in, target_out = target[:, :-1], target[:, 1:]

    # Loss
    pred = model(source, target_in)      # [batch_size, n_tgt_tokens-1, n_vocab]
    pred = pred.view(-1, pred.shape[2])  # [batch_size * (n_tgt_tokens - 1), n_vocab]
    target_out = target_out.flatten()    # [batch_size * (n_tgt_tokens - 1),]
    metrics['loss'] = loss_fn(pred, target_out)

    # Accuracy - we ignore the padding predictions
    for k in [1, 5, 10]:
        metrics[f'top-{k}'] = topk_accuracy(target_out, pred, k, config['tgt_pad_idx'])

    return metrics


def eval_model(model: nn.Module, dataloader: DataLoader, config: dict) -> dict:
    """Evaluate the model on the given dataloader.
    """
    device = config['device']
    logs = defaultdict(list)

    model.to(device)
    model.eval()

    with torch.no_grad():
        for source, target in dataloader:
            metrics = loss_batch(model, source, target, config)
            for name, value in metrics.items():
                logs[name].append(value.cpu().item())

    for name, values in logs.items():
        logs[name] = np.mean(values)
    return logs


def train_model(model: nn.Module, config: dict):
    """Train the model in a teacher forcing manner.
    """
    train_loader, val_loader = config['train_loader'], config['val_loader']
    train_dataset, val_dataset = train_loader.dataset.dataset, val_loader.dataset.dataset
    optimizer = config['optimizer']
    clip = config['clip']
    device = config['device']

    columns = ['epoch']
    for mode in ['train', 'validation']:
        columns += [
            f'{mode} - {colname}'
            for colname in ['source', 'target', 'predicted', 'likelihood']
        ]
    log_table = wandb.Table(columns=columns)


    print(f'Starting training for {config["epochs"]} epochs, using {device}.')
    for e in range(config['epochs']):
        print(f'\nEpoch {e+1}')

        model.to(device)
        model.train()
        logs = defaultdict(list)

        for batch_id, (source, target) in enumerate(train_loader):
            optimizer.zero_grad()

            metrics = loss_batch(model, source, target, config)
            loss = metrics['loss']

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            for name, value in metrics.items():
                logs[name].append(value.cpu().item())  # Don't forget the '.item' to free the cuda memory

            if batch_id % config['log_every'] == 0:
                for name, value in logs.items():
                    logs[name] = np.mean(value)

                train_logs = {
                    f'Train - {m}': v
                    for m, v in logs.items()
                }
                wandb.log(train_logs)
                logs = defaultdict(list)

        # Logs
        if len(logs) != 0:
            for name, value in logs.items():
                logs[name] = np.mean(value)
            train_logs = {
                f'Train - {m}': v
                for m, v in logs.items()
            }
        else:
            logs = {
                m.split(' - ')[1]: v
                for m, v in train_logs.items()
            }

        print_logs('Train', logs)

        logs = eval_model(model, val_loader, config)
        print_logs('Eval', logs)
        val_logs = {
            f'Validation - {m}': v
            for m, v in logs.items()
        }

        val_source, val_target = val_dataset[ torch.randint(len(val_dataset), (1,)) ]
        val_pred, val_prob = beam_search(
            model,
            val_source,
            config['src_vocab'],
            config['tgt_vocab'],
            config['src_tokenizer'],
            device,  # It can take a lot of VRAM
            beam_width=10,
            max_target=100,
            max_sentence_length=config['max_sequence_length'],
        )[0]
        print(val_source)
        print(val_pred)

        logs = {**train_logs, **val_logs}  # Merge dictionnaries
        wandb.log(logs)  # Upload to the WandB cloud

        # Table logs
        train_source, train_target = train_dataset[ torch.randint(len(train_dataset), (1,)) ]
        train_pred, train_prob = beam_search(
            model,
            train_source,
            config['src_vocab'],
            config['tgt_vocab'],
            config['src_tokenizer'],
            device,  # It can take a lot of VRAM
            beam_width=10,
            max_target=100,
            max_sentence_length=config['max_sequence_length'],
        )[0]

        data = [
            e + 1,
            train_source, train_target, train_pred, train_prob,
            val_source, val_target, val_pred, val_prob,
        ]
        log_table.add_data(*data)

    # Log the table at the end of the training
    wandb.log({'Model predictions': log_table})

# Training the models
We can now finally train the models.
Choose the right hyperparameters, play with them and try to find
ones that lead to good models and good training curves.
Try to reach a loss under 1.0.

So you know, it is possible to get descent results with approximately 20 epochs.
With CUDA enabled, one epoch, even on a big model with a big dataset, shouldn't last more than 10 minutes.
A normal epoch is between 1 to 5 minutes.

*This is considering Colab Pro, we should try using free Colab to get better estimations.*

---

To test your implementations, it is easier to try your models
in a CPU instance. Indeed, Colab reduces your GPU instances priority
with the time you recently past using GPU instances. It would be
sad to consume all your GPU time on implementation testing.
Moreover, you should try your models on small datasets and with a small number of parameters.
For exemple, you could set:
```
MAX_SEQ_LEN = 10
MIN_TOK_FREQ = 20
dim_embedding = 40
dim_hidden = 60
n_layers = 1
```

You usually don't want to log anything onto WandB when testing your implementation.
To deactivate WandB without having to change any line of code, you can type `!wandb offline` in a cell.

Once you have rightly implemented the models, you can train bigger models on bigger datasets.
When you do this, do not forget to change the runtime as GPU (and use `!wandb online`)!

In [ ]:

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!wandb login


!nvidia-smi

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Tue Apr 11 05:08:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |            

In [ ]:
# Instanciate the datasets

MAX_SEQ_LEN = 60
MIN_TOK_FREQ = 2
train_dataset, val_dataset = build_datasets(
    MAX_SEQ_LEN,
    MIN_TOK_FREQ,
    en_tokenizer,
    fr_tokenizer,
    train,
    valid,
)


print(f'English vocabulary size: {len(train_dataset.en_vocab):,}')
print(f'French vocabulary size: {len(train_dataset.fr_vocab):,}')

print(f'\nTraining examples: {len(train_dataset):,}')
print(f'Validation examples: {len(val_dataset):,}')

English vocabulary size: 11,753
French vocabulary size: 17,820

Training examples: 196,176
Validation examples: 21,797


In [ ]:
# Build the model, the dataloaders, optimizer and the loss function
# Log every hyperparameters and arguments into the config dictionnary

config = {
    # General parameters
    'epochs': 5,
    'batch_size': 128,
    'lr': 1e-3,
    'betas': (0.9, 0.99),
    'clip': 5,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',

    # Model parameters
    'n_tokens_src': len(train_dataset.en_vocab),
    'n_tokens_tgt': len(train_dataset.fr_vocab),
    'n_heads': 4,
    'dim_embedding': 196,
    'dim_hidden': 256,
    'n_layers': 3,
    'dropout': 0.1,
    'model_type': 'GRU',

    # Others
    'max_sequence_length': MAX_SEQ_LEN,
    'min_token_freq': MIN_TOK_FREQ,
    'src_vocab': train_dataset.en_vocab,
    'tgt_vocab': train_dataset.fr_vocab,
    'src_tokenizer': en_tokenizer,
    'tgt_tokenizer': fr_tokenizer,
    'src_pad_idx': train_dataset.en_vocab['<pad>'],
    'tgt_pad_idx': train_dataset.fr_vocab['<pad>'],
    'seed': 0,
    'log_every': 50,  # Number of batches between each wandb logs
}

torch.manual_seed(config['seed'])

config['train_loader'] = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,

    collate_fn=lambda batch: generate_batch(batch, config['src_pad_idx'], config['tgt_pad_idx'])
)

config['val_loader'] = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=True,


    collate_fn=lambda batch: generate_batch(batch, config['src_pad_idx'], config['tgt_pad_idx'])
)
"""
model = TranslationRNN(
    config['n_tokens_src'],
    config['n_tokens_tgt'],
    config['dim_embedding'],
    config['dim_hidden'],
    config['n_layers'],
    config['dropout'],
    config['src_pad_idx'],
    config['tgt_pad_idx'],
    config['model_type'],
)
"""
model = TranslationTransformer(
    config['n_tokens_src'],
    config['n_tokens_tgt'],
    config['n_heads'],
    config['dim_embedding'],
    config['dim_hidden'],
    config['n_layers'],
    config['dropout'],
    config['src_pad_idx'],
    config['tgt_pad_idx'],
)


config['optimizer'] = optim.Adam(
    model.parameters(),
    lr=config['lr'],
    betas=config['betas'],
)

weight_classes = torch.ones(config['n_tokens_tgt'], dtype=torch.float)
weight_classes[config['tgt_vocab']['<unk>']] = 0.1  # Lower the importance of that class
config['loss'] = nn.CrossEntropyLoss(
    weight=weight_classes,
    ignore_index=config['tgt_pad_idx'],  # We do not have to learn those
)

summary(
    model,
    input_size=[
        (config['batch_size'], config['max_sequence_length']),
        (config['batch_size'], config['max_sequence_length'])
    ],
    dtypes=[torch.long, torch.long],
    depth=3,
)

Layer (type:depth-idx)                             Output Shape              Param #
TranslationTransformer                             [128, 60, 17820]          --
├─Embedding: 1-1                                   [128, 60, 196]            2,303,588
├─Embedding: 1-2                                   [128, 60, 196]            38,416
├─Dropout: 1-3                                     [128, 60, 196]            --
├─Embedding: 1-4                                   [128, 60, 196]            3,492,720
├─Embedding: 1-5                                   [128, 60, 196]            (recursive)
├─Dropout: 1-6                                     [128, 60, 196]            --
├─Transformer: 1-7                                 [128, 60, 196]            --
│    └─TransformerEncoder: 2-1                     [128, 60, 196]            --
│    │    └─ModuleList: 3-1                        --                        768,108
│    └─TransformerDecoder: 2-2                     [128, 60, 196]            --
│  

In [ ]:
#!wandb online
!wandb login --relogin
!WANDB_MODE=online

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##RNN

In [ ]:
#RNN
!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
    train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 3.37     top-1: 0.38    top-5: 0.57    top-10: 0.66
Eval -    loss: 3.28     top-1: 0.38    top-5: 0.58    top-10: 0.66
Tom enjoys telling jokes.
Je n'ai pas l'intention de faire ça.

Epoch 2
Train -   loss: 3.18     top-1: 0.39    top-5: 0.59    top-10: 0.68
Eval -    loss: 3.12     top-1: 0.40    top-5: 0.59    top-10: 0.68
Berlin is in Germany.
Je ne sais pas quoi faire.

Epoch 3
Train -   loss: 3.06     top-1: 0.40    top-5: 0.61    top-10: 0.70
Eval -    loss: 3.04     top-1: 0.40    top-5: 0.60    top-10: 0.69
The dog drank some water and went away.
Je ne sais pas quoi faire.

Epoch 4
Train -   loss: 2.91     top-1: 0.42    top-5: 0.63    top-10: 0.71
Eval -    loss: 2.93     top-1: 0.42    top-5: 0.62    top-10: 0.70
Certain conditions may apply.
Il y a quelque chose que je veux faire.

Epoch 5
Train -   loss: 2.94     top-1: 0.42    top-5: 0.62    top-10: 0.71
Eval -    loss: 2.92     top-1: 0.42    top-5: 0.6

Train - loss,█▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇██████▇▇█████
Train - top-10,▁▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████▇██████████▇██████
Train - top-5,▁▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████▇██████
Validation - loss,█▅▃▁▁
Validation - top-1,▁▄▅██
Validation - top-10,▁▄▆██
Validation - top-5,▁▄▅██
Train - loss,2.93837
Train - top-1,0.41973
Train - top-10,0.70628


##GRU

In [ ]:
#GRU
!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
    train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


wandb: Currently logged in as: aminekhouiy (team-amine). Use `wandb login --relogin` to force relogin


Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 2.38     top-1: 0.55    top-5: 0.74    top-10: 0.79
Eval -    loss: 2.28     top-1: 0.56    top-5: 0.74    top-10: 0.79
They manage to get along without much money.
Ils ont l'intention de rester plus de temps.

Epoch 2
Train -   loss: 1.89     top-1: 0.62    top-5: 0.80    top-10: 0.85
Eval -    loss: 1.81     top-1: 0.63    top-5: 0.81    top-10: 0.85
Don't speak so fast, please.
Ne parle pas si vite, s'il te plait.

Epoch 3
Train -   loss: 1.64     top-1: 0.66    top-5: 0.84    top-10: 0.88
Eval -    loss: 1.59     top-1: 0.66    top-5: 0.84    top-10: 0.88
Stop treating me like a child.
Arrête de me mettre comme un enfant.

Epoch 4
Train -   loss: 1.46     top-1: 0.69    top-5: 0.86    top-10: 0.90
Eval -    loss: 1.47     top-1: 0.68    top-5: 0.86    top-10: 0.89
You're worried, aren't you?
Tu es inquiète, n'est-ce pas ?

Epoch 5
Train -   loss: 1.34     top-1: 0.70    top-5: 0.88    top-10: 0.91
Eval -    loss: 

Train - loss,█▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▃▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████
Train - top-10,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████████
Train - top-5,▁▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
Validation - loss,█▄▃▂▁
Validation - top-1,▁▅▆▇█
Validation - top-10,▁▅▇▇█
Validation - top-5,▁▅▆▇█
Train - loss,1.34399
Train - top-1,0.70187
Train - top-10,0.91149


##Transformer


In [ ]:
#!wandb online > /dev/null  # online / offline to activate or deactivate WandB logging

with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
    train_model(model, config)

Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 1.77     top-1: 0.65    top-5: 0.84    top-10: 0.87
Eval -    loss: 1.56     top-1: 0.67    top-5: 0.86    top-10: 0.89
I didn't know what time I was supposed to leave.
Je ne savais pas ce que j'étais censé partir.

Epoch 2
Train -   loss: 1.41     top-1: 0.70    top-5: 0.88    top-10: 0.91
Eval -    loss: 1.27     top-1: 0.72    top-5: 0.89    top-10: 0.92
We appreciate the advice.
Nous apprécions les conseils.

Epoch 3
Train -   loss: 1.24     top-1: 0.72    top-5: 0.90    top-10: 0.93
Eval -    loss: 1.15     top-1: 0.74    top-5: 0.90    top-10: 0.93
I've already paid you.
Je t'ai déjà payé.

Epoch 4
Train -   loss: 1.14     top-1: 0.74    top-5: 0.91    top-10: 0.94
Eval -    loss: 1.08     top-1: 0.75    top-5: 0.91    top-10: 0.94
You don't seem to understand.
Vous ne semblez pas comprendre.

Epoch 5
Train -   loss: 1.11     top-1: 0.75    top-5: 0.91    top-10: 0.94
Eval -    loss: 1.04     top-1: 0.76    top-

Train - loss,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
Train - top-10,▁▅▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████████████████
Train - top-5,▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████████████
Validation - loss,█▄▂▂▁
Validation - top-1,▁▅▆▇█
Validation - top-10,▁▅▇▇█
Validation - top-5,▁▅▆▇█
Train - loss,1.10562
Train - top-1,0.74527
Train - top-10,0.94005


In [ ]:
sentence = "It is possible to try your work here."

preds = beam_search(
    model,
    sentence,
    config['src_vocab'],
    config['tgt_vocab'],
    config['src_tokenizer'],
    config['device'],
    beam_width=10,
    max_target=100,
    max_sentence_length=config['max_sequence_length']
)[:5]

for i, (translation, likelihood) in enumerate(preds):
    print(f'{i}. ({likelihood*100:.5f}%) \t {translation}')

0. (19.64876%) 	 Il est possible d'essayer ton travail ici.
1. (13.85630%) 	 C'est possible d'essayer ton travail ici.
2. (9.83911%) 	 Il est possible d'essayer votre travail ici.
3. (5.54668%) 	 C'est possible d'essayer votre travail ici.
4. (0.97218%) 	 Il est possible d'essayer le travail ici.


# Questions
1. Explain the differences between Vanilla RNN, GRU-RNN, and Transformers.
2. Why is positionnal encoding necessary in Transformers and not in RNNs?
3. Describe the preprocessing process. Detail how the initial dataset is processed before being fed to the translation models.

# Small report - experiments
Once everything is working fine, you can explore aspects of these models and do some research of your own into how they behave.

For exemple, you can experiment with the hyperparameters.
What are the effect of the differents hyperparameters with the final model performance? What about training time?

What are some other metrics you could have for machine translation? Can you compute them and add them to your WandB report?

Those are only examples, you can do whatever you think will be interesting.
This part account for many points, *feel free to go wild!*

---
*Make a concise report about your experiments here.*

##RNN

In [ ]:

!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
  for batchsize in [64,128,256]:
    print('batch size',batchsize)
    config['batch_size'] = batchsize
    train_model(model, config)


W&B online. Running your script from this directory will now sync to the cloud.


batch size 64
Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 3.26     top-1: 0.38    top-5: 0.58    top-10: 0.67
Eval -    loss: 3.18     top-1: 0.39    top-5: 0.59    top-10: 0.67
I need to go to work.
Je ne sais pas comment faire ça.

Epoch 2
Train -   loss: 3.09     top-1: 0.40    top-5: 0.61    top-10: 0.69
Eval -    loss: 3.08     top-1: 0.40    top-5: 0.60    top-10: 0.69
I made that for you.
Je ne peux pas le faire.

Epoch 3
Train -   loss: 3.00     top-1: 0.41    top-5: 0.61    top-10: 0.70
Eval -    loss: 3.00     top-1: 0.41    top-5: 0.61    top-10: 0.70
I'll be back in a little bit.
Je ne sais pas ce que tu as à faire.

Epoch 4
Train -   loss: 2.95     top-1: 0.42    top-5: 0.62    top-10: 0.71
Eval -    loss: 2.95     top-1: 0.41    top-5: 0.62    top-10: 0.71
I thought you'd been killed. I'm glad I was wrong.
Je ne pense pas qu'il va pleuvoir.

Epoch 5
Train -   loss: 2.91     top-1: 0.42    top-5: 0.62    top-10: 0.71
Eval -    loss: 2.91     top-1:

Train - loss,█▇▆▅▅▅▅▅▄▄▄▃▄▄▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂
Train - top-1,▁▂▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▆▆▇▆▇▇▇█▇▇▇▇▇█████
Train - top-10,▁▂▃▃▃▄▄▄▅▅▅▆▅▆▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇██▇
Train - top-5,▁▂▃▃▃▄▄▄▄▅▅▅▅▅▆▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████
Validation - loss,█▆▅▄▄▃▃▃▂▂▁▁▁▁▁
Validation - top-1,▁▂▃▄▅▅▆▆▆▇█████
Validation - top-10,▁▃▄▅▅▆▆▆▇▇▇████
Validation - top-5,▁▃▄▅▅▅▆▆▆▇▇████
Train - loss,2.65793
Train - top-1,0.4471
Train - top-10,0.74263


In [ ]:
!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
  for nlayer in [2,4]:
    print('n_layers',nlayer)
    config['n_layers'] = nlayer
    train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


n_layers 2
Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 3.37     top-1: 0.38    top-5: 0.57    top-10: 0.66
Eval -    loss: 3.28     top-1: 0.38    top-5: 0.58    top-10: 0.66
Tom enjoys telling jokes.
Je n'ai pas l'intention de faire ça.

Epoch 2
Train -   loss: 3.18     top-1: 0.39    top-5: 0.59    top-10: 0.68
Eval -    loss: 3.12     top-1: 0.40    top-5: 0.59    top-10: 0.68
Berlin is in Germany.
Je ne sais pas quoi faire.

Epoch 3
Train -   loss: 3.06     top-1: 0.40    top-5: 0.61    top-10: 0.70
Eval -    loss: 3.04     top-1: 0.40    top-5: 0.60    top-10: 0.69
The dog drank some water and went away.
Je ne sais pas quoi faire.

Epoch 4
Train -   loss: 2.91     top-1: 0.42    top-5: 0.63    top-10: 0.71
Eval -    loss: 2.93     top-1: 0.42    top-5: 0.62    top-10: 0.70
Certain conditions may apply.
Il y a quelque chose que je veux faire.

Epoch 5
Train -   loss: 2.94     top-1: 0.42    top-5: 0.62    top-10: 0.71
Eval -    loss: 2.92     top-1: 0.42   

Train - loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███▇████████████
Train - top-10,▁▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇████████████
Train - top-5,▁▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███▇████████████
Validation - loss,█▅▄▂▂▂▁▁▁▁
Validation - top-1,▁▃▄▆▆▇▇▇██
Validation - top-10,▁▃▅▆▆▇▇███
Validation - top-5,▁▃▄▆▆▇▇███
Train - loss,2.77061
Train - top-1,0.43577
Train - top-10,0.72723


In [ ]:
!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
  for dimembedding in [100,400]:
    print('dim_embedding',dimembedding)
    config['dim_embedding'] = dimembedding
    train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


Train - loss,█▁
Train - top-1,▁█
Train - top-10,▁█
Train - top-5,▁█
Train - loss,0.8784
Train - top-1,0.77376
Train - top-10,0.95431
Train - top-5,0.93123


dim_embedding 100
Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 3.37     top-1: 0.38    top-5: 0.57    top-10: 0.66
Eval -    loss: 3.28     top-1: 0.38    top-5: 0.58    top-10: 0.66
Tom enjoys telling jokes.
Je n'ai pas l'intention de faire ça.

Epoch 2
Train -   loss: 3.18     top-1: 0.39    top-5: 0.59    top-10: 0.68
Eval -    loss: 3.12     top-1: 0.40    top-5: 0.59    top-10: 0.68
Berlin is in Germany.
Je ne sais pas quoi faire.

Epoch 3
Train -   loss: 3.06     top-1: 0.40    top-5: 0.61    top-10: 0.70
Eval -    loss: 3.04     top-1: 0.40    top-5: 0.60    top-10: 0.69
The dog drank some water and went away.
Je ne sais pas quoi faire.

Epoch 4
Train -   loss: 2.91     top-1: 0.42    top-5: 0.63    top-10: 0.71
Eval -    loss: 2.93     top-1: 0.42    top-5: 0.62    top-10: 0.70
Certain conditions may apply.
Il y a quelque chose que je veux faire.

Epoch 5
Train -   loss: 2.94     top-1: 0.42    top-5: 0.62    top-10: 0.71
Eval -    loss: 2.92     top-1: 

Train - loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███▇████████████
Train - top-10,▁▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇████████████
Train - top-5,▁▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███▇████████████
Validation - loss,█▅▄▂▂▂▁▁▁▁
Validation - top-1,▁▃▄▆▆▇▇▇██
Validation - top-10,▁▃▅▆▆▇▇███
Validation - top-5,▁▃▄▆▆▇▇███
Train - loss,2.77061
Train - top-1,0.43577
Train - top-10,0.72723


##GRU

In [ ]:
#GRU
!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
  for batchsize in [64,128,256]:
    print('batch size',batchsize)
    config['batch_size'] = batchsize
    train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


batch size 64
Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 2.38     top-1: 0.55    top-5: 0.74    top-10: 0.79
Eval -    loss: 2.28     top-1: 0.56    top-5: 0.74    top-10: 0.79
They manage to get along without much money.
Ils ont l'intention de rester plus de temps.

Epoch 2
Train -   loss: 1.89     top-1: 0.62    top-5: 0.80    top-10: 0.85
Eval -    loss: 1.81     top-1: 0.63    top-5: 0.81    top-10: 0.85
Don't speak so fast, please.
Ne parle pas si vite, s'il te plait.

Epoch 3
Train -   loss: 1.64     top-1: 0.66    top-5: 0.84    top-10: 0.88
Eval -    loss: 1.59     top-1: 0.66    top-5: 0.84    top-10: 0.88
Stop treating me like a child.
Arrête de me mettre comme un enfant.

Epoch 4
Train -   loss: 1.46     top-1: 0.69    top-5: 0.86    top-10: 0.90
Eval -    loss: 1.47     top-1: 0.68    top-5: 0.86    top-10: 0.89
You're worried, aren't you?
Tu es inquiète, n'est-ce pas ?

Epoch 5
Train -   loss: 1.34     top-1: 0.70    top-5: 0.88    top-10: 0.91
Ev

Train - loss,█▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇█▇███████████
Train - top-10,▁▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇█████████████████████
Train - top-5,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█████████████████████
Validation - loss,█▅▃▃▂▂▁▁▁▁▁▁▁▁▁
Validation - top-1,▁▄▅▆▇▇▇▇███████
Validation - top-10,▁▄▆▆▇▇▇████████
Validation - top-5,▁▄▆▆▇▇▇████████
Train - loss,0.93259
Train - top-1,0.76646
Train - top-10,0.95095


In [ ]:
#GRU
!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
  for nlayer in [2,4]:
    print('n_layers',nlayer)
    config['n_layers'] = nlayer
    train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


n_layers 2
Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 2.38     top-1: 0.55    top-5: 0.74    top-10: 0.79
Eval -    loss: 2.28     top-1: 0.56    top-5: 0.74    top-10: 0.79
They manage to get along without much money.
Ils ont l'intention de rester plus de temps.

Epoch 2
Train -   loss: 1.89     top-1: 0.62    top-5: 0.80    top-10: 0.85
Eval -    loss: 1.81     top-1: 0.63    top-5: 0.81    top-10: 0.85
Don't speak so fast, please.
Ne parle pas si vite, s'il te plait.

Epoch 3
Train -   loss: 1.64     top-1: 0.66    top-5: 0.84    top-10: 0.88
Eval -    loss: 1.59     top-1: 0.66    top-5: 0.84    top-10: 0.88
Stop treating me like a child.
Arrête de me mettre comme un enfant.

Epoch 4
Train -   loss: 1.46     top-1: 0.69    top-5: 0.86    top-10: 0.90
Eval -    loss: 1.47     top-1: 0.68    top-5: 0.86    top-10: 0.89
You're worried, aren't you?
Tu es inquiète, n'est-ce pas ?

Epoch 5
Train -   loss: 1.34     top-1: 0.70    top-5: 0.88    top-10: 0.91
Eval 

Train - loss,█▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▃▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██▇▇████████████
Train - top-10,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
Train - top-5,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇████████████████
Validation - loss,█▅▃▂▂▂▁▁▁▁
Validation - top-1,▁▄▅▆▇▇▇███
Validation - top-10,▁▄▆▇▇▇████
Validation - top-5,▁▄▆▇▇▇████
Train - loss,1.06321
Train - top-1,0.74494
Train - top-10,0.93866


In [ ]:
#GRU
!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
  for dimembedding in [100,400]:
    print('dim_embedding',dimembedding)
    config['dim_embedding'] = dimembedding
    train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


dim_embedding 100
Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 2.38     top-1: 0.55    top-5: 0.74    top-10: 0.79
Eval -    loss: 2.28     top-1: 0.56    top-5: 0.74    top-10: 0.79
They manage to get along without much money.
Ils ont l'intention de rester plus de temps.

Epoch 2
Train -   loss: 1.89     top-1: 0.62    top-5: 0.80    top-10: 0.85
Eval -    loss: 1.81     top-1: 0.63    top-5: 0.81    top-10: 0.85
Don't speak so fast, please.
Ne parle pas si vite, s'il te plait.

Epoch 3
Train -   loss: 1.64     top-1: 0.66    top-5: 0.84    top-10: 0.88
Eval -    loss: 1.59     top-1: 0.66    top-5: 0.84    top-10: 0.88
Stop treating me like a child.
Arrête de me mettre comme un enfant.

Epoch 4
Train -   loss: 1.46     top-1: 0.69    top-5: 0.86    top-10: 0.90
Eval -    loss: 1.47     top-1: 0.68    top-5: 0.86    top-10: 0.89
You're worried, aren't you?
Tu es inquiète, n'est-ce pas ?

Epoch 5
Train -   loss: 1.34     top-1: 0.70    top-5: 0.88    top-10: 0.9

Train - loss,█▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▃▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██▇▇████████████
Train - top-10,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
Train - top-5,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇████████████████
Validation - loss,█▅▃▂▂▂▁▁▁▁
Validation - top-1,▁▄▅▆▇▇▇███
Validation - top-10,▁▄▆▇▇▇████
Validation - top-5,▁▄▆▇▇▇████
Train - loss,1.06321
Train - top-1,0.74494
Train - top-10,0.93866


##

In [ ]:
# Build the model, the dataloaders, optimizer and the loss function
# Log every hyperparameters and arguments into the config dictionnary

config = {
    # General parameters
    'epochs': 5,
    'batch_size': 128,
    'lr': 1e-3,
    'betas': (0.9, 0.99),
    'clip': 5,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',

    # Model parameters
    'n_tokens_src': len(train_dataset.en_vocab),
    'n_tokens_tgt': len(train_dataset.fr_vocab),
    'n_heads': 4,
    'dim_embedding': 196,
    'dim_hidden': 256,
    'n_layers': 3,
    'dropout': 0.1,
    'model_type': 'GRU',

    # Others
    'max_sequence_length': MAX_SEQ_LEN,
    'min_token_freq': MIN_TOK_FREQ,
    'src_vocab': train_dataset.en_vocab,
    'tgt_vocab': train_dataset.fr_vocab,
    'src_tokenizer': en_tokenizer,
    'tgt_tokenizer': fr_tokenizer,
    'src_pad_idx': train_dataset.en_vocab['<pad>'],
    'tgt_pad_idx': train_dataset.fr_vocab['<pad>'],
    'seed': 0,
    'log_every': 50,  # Number of batches between each wandb logs
}

torch.manual_seed(config['seed'])

config['train_loader'] = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,

    collate_fn=lambda batch: generate_batch(batch, config['src_pad_idx'], config['tgt_pad_idx'])
)

config['val_loader'] = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=True,


    collate_fn=lambda batch: generate_batch(batch, config['src_pad_idx'], config['tgt_pad_idx'])
)
"""
model = TranslationRNN(
    config['n_tokens_src'],
    config['n_tokens_tgt'],
    config['dim_embedding'],
    config['dim_hidden'],
    config['n_layers'],
    config['dropout'],
    config['src_pad_idx'],
    config['tgt_pad_idx'],
    config['model_type'],
)
"""
model = TranslationTransformer(
    config['n_tokens_src'],
    config['n_tokens_tgt'],
    config['n_heads'],
    config['dim_embedding'],
    config['dim_hidden'],
    config['n_layers'],
    config['dropout'],
    config['src_pad_idx'],
    config['tgt_pad_idx'],
)


config['optimizer'] = optim.Adam(
    model.parameters(),
    lr=config['lr'],
    betas=config['betas'],
)

weight_classes = torch.ones(config['n_tokens_tgt'], dtype=torch.float)
weight_classes[config['tgt_vocab']['<unk>']] = 0.1  # Lower the importance of that class
config['loss'] = nn.CrossEntropyLoss(
    weight=weight_classes,
    ignore_index=config['tgt_pad_idx'],  # We do not have to learn those
)

summary(
    model,
    input_size=[
        (config['batch_size'], config['max_sequence_length']),
        (config['batch_size'], config['max_sequence_length'])
    ],
    dtypes=[torch.long, torch.long],
    depth=3,
)

Layer (type:depth-idx)                             Output Shape              Param #
TranslationTransformer                             [128, 60, 17820]          --
├─Embedding: 1-1                                   [128, 60, 196]            2,303,588
├─Embedding: 1-2                                   [128, 60, 196]            38,416
├─Dropout: 1-3                                     [128, 60, 196]            --
├─Embedding: 1-4                                   [128, 60, 196]            3,492,720
├─Embedding: 1-5                                   [128, 60, 196]            (recursive)
├─Dropout: 1-6                                     [128, 60, 196]            --
├─Transformer: 1-7                                 [128, 60, 196]            --
│    └─TransformerEncoder: 2-1                     [128, 60, 196]            --
│    │    └─ModuleList: 3-1                        --                        768,108
│    └─TransformerDecoder: 2-2                     [128, 60, 196]            --
│  

##Transformer


In [ ]:
#tranasformer
!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
  for batchsize in [64,256]:
    print('batch size',batchsize)
    config['batch_size'] = batchsize
    train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


batch size 64
Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 1.05     top-1: 0.75    top-5: 0.92    top-10: 0.94
Eval -    loss: 1.00     top-1: 0.77    top-5: 0.92    top-10: 0.94
Billie Holliday had an earthy, gravelly voice.
Le <unk> a eu une voix <unk>.

Epoch 2
Train -   loss: 0.99     top-1: 0.76    top-5: 0.93    top-10: 0.95
Eval -    loss: 0.98     top-1: 0.77    top-5: 0.93    top-10: 0.95
Don't just stand there, say something.
Ne restez pas planté là-bas, dis quelque chose.

Epoch 3
Train -   loss: 0.97     top-1: 0.77    top-5: 0.93    top-10: 0.95
Eval -    loss: 0.95     top-1: 0.77    top-5: 0.93    top-10: 0.95
I was really quite stiff.
J'étais vraiment tout à fait mal.

Epoch 4
Train -   loss: 0.93     top-1: 0.78    top-5: 0.93    top-10: 0.95
Eval -    loss: 0.93     top-1: 0.78    top-5: 0.93    top-10: 0.95
I was a fool to trust you.
J'ai été idiot de te faire confiance.

Epoch 5
Train -   loss: 0.89     top-1: 0.78    top-5: 0.94    top-10: 0

Train - loss,▇███▅▆▇▇▄▆▆▆▄▅▆▅▃▄▅▅▃▄▅▅▂▃▄▄▂▃▄▄▁▂▃▄▁▂▃▄
Train - top-1,▂▂▁▂▄▂▂▂▅▄▃▄▅▄▄▄▆▅▅▅▆▅▅▅▇▆▅▅█▆▆▅▇▇▆▆█▇▆▆
Train - top-10,▂▁▁▁▄▃▂▁▅▃▃▃▅▅▃▄▆▅▄▄▆▅▄▄▇▆▅▅▇▆▅▅▇▇▆▅█▇▆▅
Train - top-5,▂▁▁▁▄▃▂▂▅▃▃▃▅▄▃▃▆▅▄▄▆▅▄▄▇▆▅▅▇▆▅▅▇▇▆▆█▇▆▅
Validation - loss,█▇▅▄▄▃▂▂▂▁
Validation - top-1,▁▂▃▅▅▆▇█▇█
Validation - top-10,▁▃▄▅▅▆▇▇▇█
Validation - top-5,▁▃▄▅▅▆▇▇▇█
Train - loss,0.82515
Train - top-1,0.78987
Train - top-10,0.96311


In [ ]:
#Transformer
!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
  for nlayer in [2,4]:
    print('n_layers',nlayer)
    config['n_layers'] = nlayer
    train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


n_layers 2
Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 1.77     top-1: 0.65    top-5: 0.84    top-10: 0.87
Eval -    loss: 1.56     top-1: 0.67    top-5: 0.86    top-10: 0.89
I didn't know what time I was supposed to leave.
Je ne savais pas ce que j'étais censé partir.

Epoch 2
Train -   loss: 1.41     top-1: 0.70    top-5: 0.88    top-10: 0.91
Eval -    loss: 1.27     top-1: 0.72    top-5: 0.89    top-10: 0.92
We appreciate the advice.
Nous apprécions les conseils.

Epoch 3
Train -   loss: 1.24     top-1: 0.72    top-5: 0.90    top-10: 0.93
Eval -    loss: 1.15     top-1: 0.74    top-5: 0.90    top-10: 0.93
I've already paid you.
Je t'ai déjà payé.

Epoch 4
Train -   loss: 1.14     top-1: 0.74    top-5: 0.91    top-10: 0.94
Eval -    loss: 1.08     top-1: 0.75    top-5: 0.91    top-10: 0.94
You don't seem to understand.
Vous ne semblez pas comprendre.

Epoch 5
Train -   loss: 1.11     top-1: 0.75    top-5: 0.91    top-10: 0.94
Eval -    loss: 1.04     top-1: 0

Train - loss,█▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇████████████
Train - top-10,▁▄▅▆▆▇▇▇▇▇▇▇▇▇▇▇██▇▇████████████████████
Train - top-5,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
Validation - loss,█▅▃▃▂▂▂▁▁▁
Validation - top-1,▁▄▅▆▇▇▇███
Validation - top-10,▁▄▆▆▇▇▇███
Validation - top-5,▁▄▆▆▇▇▇███
Train - loss,0.88728
Train - top-1,0.78144
Train - top-10,0.95853


In [ ]:
#Transformer
!wandb online
with wandb.init(
        config=config,
        project='INF8225 - TP3',  # Title of your project

        group='Transformer - small',  # In what group of runs do you want this run to be in?
        save_code=True,

    ):
  for dimembedding in [100,400]:
    print('dim_embedding',dimembedding)
    config['dim_embedding'] = dimembedding
    train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


dim_embedding 100
Starting training for 5 epochs, using cuda.

Epoch 1
Train -   loss: 1.77     top-1: 0.65    top-5: 0.84    top-10: 0.87
Eval -    loss: 1.56     top-1: 0.67    top-5: 0.86    top-10: 0.89
I didn't know what time I was supposed to leave.
Je ne savais pas ce que j'étais censé partir.

Epoch 2
Train -   loss: 1.41     top-1: 0.70    top-5: 0.88    top-10: 0.91
Eval -    loss: 1.27     top-1: 0.72    top-5: 0.89    top-10: 0.92
We appreciate the advice.
Nous apprécions les conseils.

Epoch 3
Train -   loss: 1.24     top-1: 0.72    top-5: 0.90    top-10: 0.93
Eval -    loss: 1.15     top-1: 0.74    top-5: 0.90    top-10: 0.93
I've already paid you.
Je t'ai déjà payé.

Epoch 4
Train -   loss: 1.14     top-1: 0.74    top-5: 0.91    top-10: 0.94
Eval -    loss: 1.08     top-1: 0.75    top-5: 0.91    top-10: 0.94
You don't seem to understand.
Vous ne semblez pas comprendre.

Epoch 5
Train -   loss: 1.11     top-1: 0.75    top-5: 0.91    top-10: 0.94
Eval -    loss: 1.04     t

Train - loss,█▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇████████████
Train - top-10,▁▄▅▆▆▇▇▇▇▇▇▇▇▇▇▇██▇▇████████████████████
Train - top-5,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
Validation - loss,█▅▃▃▂▂▂▁▁▁
Validation - top-1,▁▄▅▆▇▇▇███
Validation - top-10,▁▄▆▆▇▇▇███
Validation - top-5,▁▄▆▆▇▇▇███
Train - loss,0.88728
Train - top-1,0.78144
Train - top-10,0.95853
